<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/BOB_MVP_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install flask -q
!pip install flask_migrate -q
!pip install flask_sqlalchemy -q
!pip install anthropic -q
!pip install colab-env --quiet

## anthropic setup

In [3]:
import anthropic
import os
import colab_env
import json

api_key = os.environ["CLAUDE3_API_KEY"]
model="claude-3-7-sonnet-20250219", # 30/04/2005

client = anthropic.Anthropic(
    api_key=api_key,
)

Mounted at /content/gdrive


In [4]:
import secrets

secret_key = secrets.token_urlsafe(32)
print(secret_key)

qdTMI3weT6i0uaG2i6PJKMy6dTn0b5an1sI9SmV5yxU


## TEST CODE

In [ ]:
import colab_env

In [ ]:
import unittest
from flask import Flask, request, jsonify
from flask_sqlalchemy import SQLAlchemy
from werkzeug.security import generate_password_hash, check_password_hash
from datetime import datetime, timedelta
import jwt
import anthropic  # Make sure you have installed anthropic: !pip install anthropic
import os
import secrets
import threading
from functools import wraps
import json


# Generate a secure secret key if not already set
if 'FLASK_SECRET_KEY' not in os.environ:
    secret_key = secrets.token_urlsafe(32)
    print(f"Generated secret key: {secret_key}")
    os.environ['FLASK_SECRET_KEY'] = secret_key

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite://'  # In-memory database
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
app.config['SECRET_KEY'] = os.environ.get('FLASK_SECRET_KEY')

if not app.config['SECRET_KEY']:
    raise ValueError("FLASK_SECRET_KEY environment variable is not set")

db = SQLAlchemy(app)

class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    user_type = db.Column(db.String(50), nullable=False)
    email = db.Column(db.String(100), unique=True, nullable=False)
    password_hash = db.Column(db.String(128), nullable=False)
    first_name = db.Column(db.String(50))
    last_name = db.Column(db.String(50))
    created_at = db.Column(db.DateTime, default=datetime.utcnow)

    def set_password(self, password):
        self.password_hash = generate_password_hash(password)

    def check_password(self, password):
        return check_password_hash(self.password_hash, password)

    def __repr__(self):
        return f'<User {self.email}>'

class Itinerary(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    user_id = db.Column(db.Integer, db.ForeignKey('user.id'), nullable=False)
    title = db.Column(db.String(200), nullable=False)
    start_date = db.Column(db.DateTime)
    end_date = db.Column(db.DateTime)
    user = db.relationship('User', backref=db.backref('itineraries', lazy=True))

    def __repr__(self):
        return f'<Itinerary {self.title}>'

class Activity(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    provider_id = db.Column(db.Integer, db.ForeignKey('user.id'), nullable=False)
    name = db.Column(db.String(200), nullable=False)
    description = db.Column(db.Text)
    location = db.Column(db.String(100))
    price = db.Column(db.Float)
    provider = db.relationship('User', backref=db.backref('activities', lazy=True))

    def __repr__(self):
        return f'<Activity {self.name}>'

class Payment(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    user_id = db.Column(db.Integer, db.ForeignKey('user.id'), nullable=False)
    amount = db.Column(db.Float, nullable=False)
    payment_date = db.Column(db.DateTime, default=datetime.utcnow, nullable=False)
    payment_method = db.Column(db.String(50), nullable=False)
    transaction_id = db.Column(db.String(100), unique=True, nullable=False)
    user = db.relationship('User', backref=db.backref('payments', lazy=True))

    def __repr__(self):
        return f'<Payment {self.id}>'

class Review(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    user_id = db.Column(db.Integer, db.ForeignKey('user.id'), nullable=False)
    activity_id = db.Column(db.Integer, db.ForeignKey('activity.id'), nullable=False)
    rating = db.Column(db.Integer, nullable=False)  # e.g., 1 to 5
    comment = db.Column(db.Text, nullable=False)
    review_date = db.Column(db.DateTime, default=datetime.utcnow, nullable=False)
    user = db.relationship('User', backref=db.backref('reviews', lazy=True))
    activity = db.relationship('Activity', backref=db.backref('reviews', lazy=True))

    def __repr__(self):
        return f'<Review {self.id}>'

# --- token_required decorator ---
def token_required(f):
    @wraps(f)
    def decorated(*args, **kwargs):
        token = request.headers.get('Authorization')
        if not token:
            return jsonify({'message': 'Token is missing!'}), 401

        try:
            data = jwt.decode(token, app.config['SECRET_KEY'], algorithms=["HS256"])
            current_user = db.session.get(User, data['user_id'])  # Use db.session.get()
        except jwt.ExpiredSignatureError:
            return jsonify({'message': 'Token has expired!'}), 401
        except jwt.InvalidTokenError:
            return jsonify({'message': 'Invalid token!'}), 401
        except Exception as e:
            print(f"Error decoding token: {e}")
            return jsonify({'message': 'An error occurred while decoding the token!'}), 401

        return f(current_user, *args, **kwargs)
    return decorated

# --- register_user, login routes ---
@app.route('/register', methods=['POST'])
def register_user():
    data = request.get_json()
    if not data:
        return jsonify({'message': 'No data provided'}), 400

    required_fields = ('user_type', 'email', 'password')
    if not all(k in data for k in required_fields):
        return jsonify({'message': 'Missing required fields'}), 400

    user_type = data['user_type']
    email = data['email']
    password = data['password']
    first_name = data.get('first_name')
    last_name = data.get('last_name')

    if User.query.filter_by(email=email).first():
        return jsonify({'message': 'Email already exists'}), 400

    # ==============================================================
    # Modification: Removed 'method='sha256'' from generate_password_hash
    hashed_password = generate_password_hash(password)
    # ==============================================================

    new_user = User(user_type=user_type, email=email, password_hash=hashed_password,
                    first_name=first_name, last_name=last_name)
    db.session.add(new_user)
    try:
        db.session.commit()
    except Exception as e:
        db.session.rollback()
        print(f"Error during registration: {e}")
        return jsonify({'message': 'Failed to register user'}), 500

    return jsonify({'message': 'Registered successfully!'}), 201

@app.route('/login', methods=['POST'])
def login():
    auth = request.authorization

    if not auth or not auth.username or not auth.password:
        return jsonify({'message': 'Could not verify'}), 401, {'WWW-Authenticate': 'Basic realm="Login Required!"'}

    user = User.query.filter_by(email=auth.username).first()

    if not user or not check_password_hash(user.password_hash, auth.password):
        return jsonify({'message': 'Could not verify'}), 401, {'WWW-Authenticate': 'Basic realm="Login Required!"'}

    token = jwt.encode(
        {'user_id': user.id, 'exp': datetime.utcnow() + timedelta(hours=24)},
        app.config['SECRET_KEY'],
        algorithm="HS256"
    )
    return jsonify({'token': token, 'user_id': user.id, 'user_type': user.user_type}), 200

# --- other routes: /itineraries, /activities, /payments, /reviews ---
# --- Itinerary Endpoints ---
@app.route('/itineraries', methods=['POST'])
@token_required
def create_itinerary(current_user):
    data = request.get_json()
    if not data:
        return jsonify({'message': 'No data provided'}), 400
    if not data.get('title'):
        return jsonify({'message': 'Title is required'}), 400

    new_itinerary = Itinerary(
        user_id=current_user.id,
        title=data['title'],
        start_date=data.get('start_date'),
        end_date=data.get('end_date')
    )
    db.session.add(new_itinerary)
    try:
        db.session.commit()
    except Exception as e:
        db.session.rollback()
        print(f"Error creating itinerary: {e}")
        return jsonify({'message': 'Failed to create itinerary'}), 500
    return jsonify({'message': 'Itinerary created!'}), 201

@app.route('/itineraries', methods=['GET'])
@token_required
def get_itineraries(current_user):
    itineraries = Itinerary.query.filter_by(user_id=current_user.id).all()
    output = [{
        'id': itinerary.id,
        'title': itinerary.title,
        'start_date': itinerary.start_date,
        'end_date': itinerary.end_date
    } for itinerary in itineraries]
    return jsonify({'itineraries': output}), 200

@app.route('/itineraries/<int:itinerary_id>', methods=['GET'])
@token_required
def get_itinerary(current_user, itinerary_id):
    itinerary = Itinerary.query.filter_by(user_id=current_user.id, id=itinerary_id).first()
    if not itinerary:
        return jsonify({'message': 'Itinerary not found'}), 404
    output = {
        'id': itinerary.id,
        'title': itinerary.title,
        'start_date': itinerary.start_date,
        'end_date': itinerary.end_date
    }
    return jsonify(output), 200

# --- Activities Endpoints ---
@app.route('/activities', methods=['POST'])
@token_required
def create_activity(current_user):
    if current_user.user_type != 'provider':
        return jsonify({'message': 'Only providers can create activities'}), 403

    data = request.get_json()
    if not data:
        return jsonify({'message': 'No data provided'}), 400

    if not all(k in data for k in ('name', 'description', 'location', 'price')):
        return jsonify({'message': 'Missing required fields'}), 400

    name = data['name']
    description = data['description']
    location = data['location']
    price = data['price']
    new_activity = Activity(provider_id=current_user.id, name=name, description=description,
                            location=location, price=price)
    db.session.add(new_activity)
    try:
        db.session.commit()
    except Exception as e:
        db.session.rollback()
        print(f"Error creating activity: {e}")
        return jsonify({'message': 'Failed to create activity'}), 500
    return jsonify({'message': 'Activity created!'}), 201

@app.route('/activities', methods=['GET'])
def get_activities():
    activities = Activity.query.all()
    output = [{
        'id': activity.id,
        'name': activity.name,
        'description': activity.description,
        'location': activity.location,
        'price': activity.price
    } for activity in activities]
    return jsonify({'activities': output}), 200

@app.route('/activities/<int:activity_id>', methods=['GET'])
def get_activity(activity_id):
    activity = Activity.query.get(activity_id)
    if not activity:
        return jsonify({'message': 'Activity not found'}), 404
    output = {
        'id': activity.id,
        'name': activity.name,
        'description': activity.description,
        'location': activity.location,
        'price': activity.price
    }
    return jsonify(output), 200

# --- Payment Endpoints ---
@app.route('/payments', methods=['POST'])
@token_required
def create_payment(current_user):
    data = request.get_json()
    if not data:
        return jsonify({'message': 'No data provided'}), 400

    required_fields = ('amount', 'payment_method', 'transaction_id')
    if not all(k in data for k in required_fields):
        return jsonify({'message': 'Missing required fields'}), 400

    amount = data['amount']
    payment_method = data['payment_method']
    transaction_id = data['transaction_id']

    # Assume payment processing is successful for this example
    payment_successful = True

    if payment_successful:
        new_payment = Payment(
            user_id=current_user.id,
            amount=amount,
            payment_method=payment_method,
            transaction_id=transaction_id,
        )
        db.session.add(new_payment)
        try:
            db.session.commit()
        except Exception as e:
            db.session.rollback()
            print(f"Error creating payment: {e}")
            return jsonify({'message': 'Failed to create payment'}), 500
        return jsonify({'message': 'Payment successful!'}), 201
    else:
        return jsonify({'message': 'Payment failed'}), 400

@app.route('/payments', methods=['GET'])
@token_required
def get_payments(current_user):
    payments = Payment.query.filter_by(user_id=current_user.id).all()
    output = [{
        'id': payment.id,
        'amount': payment.amount,
        'payment_date': payment.payment_date,
        'payment_method': payment.payment_method,
        'transaction_id': payment.transaction_id
    } for payment in payments]
    return jsonify({'payments': output}), 200

# --- Review Endpoints ---
@app.route('/reviews', methods=['POST'])
@token_required
def create_review(current_user):
    data = request.get_json()
    if not data:
        return jsonify({'message': 'No data provided'}), 400

    required_fields = ('activity_id', 'rating', 'comment')
    if not all(k in data for k in required_fields):
        return jsonify({'message': 'Missing required fields'}), 400

    activity_id = data['activity_id']
    rating = data['rating']
    comment = data['comment']

    # Check if the activity exists
    activity = Activity.query.get(activity_id)
    if not activity:
        return jsonify({'message': 'Activity not found'}), 404

    # Check if the user has already reviewed this activity
    existing_review = Review.query.filter_by(user_id=current_user.id, activity_id=activity_id).first()
    if existing_review:
        return jsonify({'message': 'You have already reviewed this activity'}), 400

    new_review = Review(
        user_id=current_user.id,
        activity_id=activity_id,
        rating=rating,
        comment=comment,
    )
    db.session.add(new_review)
    try:
        db.session.commit()
    except Exception as e:
        db.session.rollback()
        print(f"Error creating review: {e}")
        return jsonify({'message': 'Failed to create review'}), 500
    return jsonify({'message': 'Review created!'}), 201

@app.route('/reviews/<int:activity_id>', methods=['GET'])
def get_reviews_for_activity(activity_id):
    reviews = Review.query.filter_by(activity_id=activity_id).all()
    if not reviews:
        return jsonify({'message': 'No reviews found for this activity'}), 404  # Changed to 404

    output = [{
        'id': review.id,
        'user_id': review.user_id,
        'rating': review.rating,
        'comment': review.comment,
        'review_date': review.review_date
    } for review in reviews]
    return jsonify({'reviews': output}), 200

# --- Asynchronous LLM Calls ---
import os
import anthropic
import threading

from queue import Queue
from functools import wraps


# Create a queue to store the LLM response
llm_response_queue = Queue()

def run_async(func):
    """Decorator to run a function asynchronously in a separate thread."""
    @wraps(func)
    def async_func(*args, **kwargs):
        def worker():
            result = func(*args, **kwargs)
            llm_response_queue.put(result)  # Put the result in the queue
        func_hl = threading.Thread(target=worker)
        func_hl.start()
        return func_hl
    return async_func

@run_async
def generate_itinerary_with_llm_task(current_user, preferences, location, start_date, end_date, activities):
    """This function makes the LLM API call and returns the response."""
    try:
        print("Starting LLM task...")

        api_key = os.environ.get("CLAUDE3_API_KEY")
        if not api_key:
            raise ValueError("CLAUDE3_API_KEY environment variable not set.")

        client = anthropic.Anthropic(api_key=api_key)

        # Improved prompt with placeholders for location, start_date, end_date, activities
        prompt = """
How do you plan out your trip?

Here's a scenario:

Bob is traveling from YVR (Vancouver International Airport) to SAT (San Antonio International Airport).

Trip Details:

1. He has a 6-hour layover in DFW (Dallas/Fort Worth International Airport).
2. He has a total budget of $5000.00 for the entire trip, including meals.
3. He enjoys hiking, visiting museums, and trying new foods.
4. He is traveling with only a carry-on bag.

The Question:

Considering Bob's interests, budget, and layover time, what are some recommendations for activities he can do during his 6-hour layover in DFW? Please suggest specific activities and include estimated times for each activity.

Important Considerations:

* Time: Bob has a limited time window and needs to account for travel time to and from the airport.
* Budget: Suggest activities that are within his budget.
* Interests: Focus on activities that align with Bob's interests in hiking, museums, and food.
* Luggage: Keep in mind that he only has a carry-on bag.

Desired Output:

Provide a structured response in JSON format with a list of recommended activities, including:

* "Activity Name": The name of the activity.
* "Description": A brief description of the activity.
* "Estimated Time": The approximate time required for the activity, including travel time.
* "Estimated Cost": The approximate cost of the activity.

## JSON Response Format:
"""

        print(prompt)  # Print the prompt for debugging
        print("Sending request to LLM...")

        response = client.completions.create(
            model="claude-3-7-sonnet-20250219",
            max_tokens_to_sample=2000,
            prompt=prompt,
        )

        print("LLM call completed")
        print(f"LLM response: {response.completion}")

        # Put the response in the queue (if using a queue)
        llm_response_queue.put(response.completion)

        return response.completion

    except Exception as e:
        print(f"Error in LLM task: {e}")
        llm_response_queue.put(None)  # Signal an error
        return None

@app.route('/generate_itinerary', methods=['POST'])
@token_required
def generate_itinerary_with_llm(current_user):
    """Endpoint handler for itinerary generation."""
    data = request.get_json()
    if not data:
        return jsonify({'message': 'No data provided'}), 400

    # Get itinerary details from the request data
    preferences = data.get('preferences')
    location = data.get('location')
    start_date = data.get('start_date')
    end_date = data.get('end_date')
    activities = data.get('activities')

    if not all([preferences, location, start_date, end_date, activities]):
        return jsonify({'message': 'Missing required fields (preferences, location, start_date, end_date, activities)'}), 400

    # Start the asynchronous LLM task
    llm_response_thread = generate_itinerary_with_llm_task(current_user, preferences, location, start_date, end_date, activities)

    # Wait for the thread to complete and get the result from the queue
    llm_response_thread.join()
    llm_response = llm_response_queue.get()

    if llm_response is not None:
        try:
            # Parse the LLM response as JSON
            itinerary_details = json.loads(llm_response)

            # Create itinerary data with datetime objects (if start_date and end_date are strings)
            itinerary_data = {
                'title': itinerary_details.get('title', 'Generated Itinerary'),
                'description': itinerary_details.get('description', 'No description available'),
                'days': itinerary_details.get('days', []), # Assuming 'days' is the key in LLM response
                'start_date': datetime.strptime(start_date, '%Y-%m-%d') if isinstance(start_date, str) else start_date,
                'end_date': datetime.strptime(end_date, '%Y-%m-%d') if isinstance(end_date, str) else end_date
            }

            # --- Itinerary Creation and Storage Logic ---
            new_itinerary = Itinerary(
                user_id=current_user.id,
                title=itinerary_data['title'],
                start_date=itinerary_data['start_date'],
                end_date=itinerary_data['end_date']
            )
            db.session.add(new_itinerary)
            db.session.commit()

            # --- Logic to store activity details (example using a separate Activity model) ---
            for day_data in itinerary_data['days']:
                day_number = day_data.get('day', 1)  # Get day number, default to 1
                for activity_data in day_data.get('activities', []):
                    # Clean the cost string (remove $, commas, etc.)
                    cost_str = activity_data.get('cost', '0').replace('$', '').replace(',', '')

                    # Convert to float
                    try:
                        cost = float(cost_str)
                    except ValueError:
                        cost = 0.0  # Default to 0 if conversion fails

                    new_activity = Activity(
                        provider_id=current_user.id,  # Or assign to a specific provider if needed
                        name=activity_data.get('name', 'Unnamed Activity'),
                        description=activity_data.get('description', ''),
                        location=activity_data.get('location', location),  # Use itinerary location if not specified
                        price=cost,  # Assign the cleaned cost value
                    )
                    db.session.add(new_activity)
            db.session.commit()

            return jsonify({'message': 'Itinerary generation request submitted', 'itinerary_id': new_itinerary.id}), 202

        except (json.JSONDecodeError, KeyError) as e:
            # --- Error Handling ---
            print(f"Error processing LLM response: {e}")  # Log the error
            db.session.rollback()  # Rollback the database session in case of errors
            return jsonify({'message': 'Failed to process itinerary details'}), 500

    else:
        return jsonify({'message': 'Failed to generate itinerary'}), 500

# --- Error Handling ---
@app.errorhandler(404)
def not_found(error):
    return jsonify({'error': 'Not found'}), 404

@app.errorhandler(500)
def internal_server_error(error):
    db.session.rollback()
    print(f"Internal Server Error: {error}")  # Log the error
    return jsonify({'error': 'Internal server error'}), 500


# --- Test Cases (using unittest) ---
from unittest.mock import patch
class TestUserRegistration(unittest.TestCase):
    def setUp(self):
        self.app = app.test_client()
        self.app_context = app.app_context()  # Create an application context
        self.app_context.push()               # Push the context
        with app.app_context():
            db.create_all()

    def tearDown(self):
        db.session.remove()
        db.drop_all()
        self.app_context.pop()               # Pop the context after the test

    def test_register_user(self):
        response = self.app.post('/register', json={'user_type': 'free', 'email': 'test@example.com', 'password': 'password123'})
        self.assertEqual(response.status_code, 201)
        self.assertIn('message', response.json)
        self.assertEqual(response.json['message'], 'Registered successfully!')

        # Now you are within the application context
        with app.app_context():
            user = User.query.filter_by(email='test@example.com').first()

        self.assertIsNotNone(user)
        self.assertEqual(user.email, 'test@example.com')
        self.assertEqual(user.user_type, 'free')
        print("Running test: test_register_user")  # Print test name


    def test_register_user_missing_data(self):
        response = self.app.post('/register', json={'user_type': 'free', 'password': 'password123'})  # Missing email
        self.assertEqual(response.status_code, 400)
        self.assertIn('message', response.json)
        self.assertEqual(response.json['message'], 'Missing required fields')
        print("Running test: test_register_user_missing_data")

    def test_register_user_duplicate_email(self):
        # First, register a user
        self.app.post('/register', json={'user_type': 'free', 'email': 'test@example.com', 'password': 'password123'})
        print("Running test: test_register_user_duplicate_email")  # Print test name


        # Try to register another user with the same email
        response = self.app.post('/register', json={'user_type': 'paid', 'email': 'test@example.com', 'password': 'anotherpassword'})
        self.assertEqual(response.status_code, 400)
        self.assertIn('message', response.json)
        self.assertEqual(response.json['message'], 'Email already exists')

    @patch('anthropic.Anthropic.completions')  # Mock the Anthropic API call
    def test_generate_itinerary_with_llm(self, mock_completions):
        """Test itinerary generation with LLM"""

        # Mock the response from the Anthropic API
        mock_completions.create.return_value.completion = json.dumps({
            "title": "Test Itinerary",
            "description": "A test itinerary generated by the LLM.",
            "days": [  # Include 'days' key in mock response
                {
                    "day": 1,
                    "activities": [
                        {"name": "Activity 1", "description": "Description 1", "duration": "2 hours", "cost": "$20"},
                        {"name": "Activity 2", "description": "Description 2", "duration": "3 hours", "cost": "$30"}
                    ]
                },
                {
                    "day": 2,
                    "activities": [
                        {"name": "Activity 3", "description": "Description 3", "duration": "4 hours", "cost": "$40"}
                    ]
                }
            ]
        })

        # Create a test user and get a token
        with app.app_context():
            user = User(user_type='free', email='test@example.com', password_hash=generate_password_hash('password123'))
            db.session.add(user)
            db.session.commit()
            token = jwt.encode({'user_id': user.id, 'exp': datetime.utcnow() + timedelta(hours=24)},
                              app.config['SECRET_KEY'], algorithm="HS256")

        # Send a POST request to the /generate_itinerary route with required data
        response = self.app.post('/generate_itinerary',
                                json={
                                    'preferences': 'Test preferences',
                                    'location': 'London',  # Add location
                                    'start_date': '2024-12-25',  # Add start_date
                                    'end_date': '2024-12-28',  # Add end_date
                                    'activities': 'sightseeing, museums, food'  # Add activities
                                },
                                headers={'Authorization': token})

        # Check the response
        self.assertEqual(response.status_code, 202)  # Expecting 202 Accepted
        self.assertIn('message', response.json)
        self.assertEqual(response.json['message'], 'Itinerary generation request submitted')

        # Check if the itinerary was created in the database
        with app.app_context():
            itinerary = Itinerary.query.filter_by(user_id=user.id, title="Test Itinerary").first()
            self.assertIsNotNone(itinerary)

            # Check if activities were created and linked to the itinerary
            activities = Activity.query.filter_by(provider_id=user.id).all()
            self.assertEqual(len(activities), 3)  # Expecting 3 activities



if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)